In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv("../data/raw/202008-divvy-tripdata.csv")

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
df.isna().sum()

In [ ]:
(df.isna().sum() / len(df))*100

In [ ]:
df_nulls = df.isna().sum()
df_nulls[df_nulls > 0].sort_values(ascending = False)

Faz sentido ter valores nulos?
de acordo com a descrição do problema as bicicletas só podem ser coletadas em um posto de bicicletas e entregues lá, portanto não há interpretação em relação à valores ausentes.
Sabendo disso , basta removermos os valore nulos

No entanto vale a pena anlisar Lat e Long , visto que pode ser que o nome da estação esteja ausente , mas a posição dele "entrega" a estação

In [ ]:
cols_coord = ['start_lat', 'start_lng', 'end_lat', 'end_lng']
mask_nulls = df.isna().any(axis=1)

df_nulls = df[mask_nulls]
df_coords_mantain = df.dropna(subset=cols_coord)
df = df[~mask_nulls]

In [ ]:
df_nulls.head()

In [ ]:
df.head()

In [ ]:
df_coords_mantain.head(10)

In [ ]:
qt_removed_lines = len(df_nulls)
percent_removed_lines = (len(df_nulls)/len(df))*100
qt_lines_coords_exists = len(df_coords_mantain) - len(df)

print(f"We removed {qt_removed_lines} lines , which represents {percent_removed_lines}% of our dataset")
print(f"But , we have {qt_lines_coords_exists} lines that have Lat and Long , but don't have Station name (start or end)")

#### 

Sabendo dessa informação , vamos separar em um df a parte apenas as estações

In [ ]:
df_stations = df[["start_station_name", "start_station_id" , "end_station_name" , "end_station_id" , "start_lat" , "start_lng" , "end_lat" , "end_lng"]]
df_start_station = df_stations[["start_station_name" , "start_station_id" , "start_lat" , "start_lng"]]
df_end_station = df_stations[["end_station_name" , "end_station_id" , "end_lat" , "end_lng"]]

In [ ]:
df_start_station = df_start_station.drop_duplicates(subset="start_station_id")
df_start_station.columns = ["station" , "id" , "lat" , "long"]
df_start_station.head()

In [ ]:
df_end_station = df_end_station.drop_duplicates(subset="end_station_id")
df_end_station.columns = ["station" , "id" , "lat" , "long"]
df_end_station.head()

In [ ]:
print(f"stations start = {len(df_start_station)} and stations end = {len(df_end_station)}")

In [ ]:
frames = [df_start_station , df_end_station]
df_stations = pd.concat(frames)
len(df_stations)

In [ ]:
df_stations.head()

In [ ]:
df_stations = df_stations.drop_duplicates(subset="id")

In [ ]:
decimal = 3

df_stations["lat"] = df_stations["lat"].round(decimal)
df_stations["long"] = df_stations["long"].round(decimal)

In [ ]:
# Essa é minha "DIM_STATIONS"
df_stations.head()

In [ ]:
decimal = 3
df_coords_mantain = df_coords_mantain[["ride_id" , "rideable_type" , "started_at" , "ended_at" , "start_lat" , "start_lng" , "end_lat" , "end_lng"]]
df_coords_mantain['start_lat_arred'] = df_coords_mantain['start_lat'].round(decimal)
df_coords_mantain['start_lng_arred'] = df_coords_mantain['start_lng'].round(decimal)
df_coords_mantain['end_lat_arred'] = df_coords_mantain['end_lat'].round(decimal)
df_coords_mantain['end_lng_arred'] = df_coords_mantain['end_lng'].round(decimal)

In [ ]:
df_coords_mantain

In [ ]:
df_resultado = pd.merge(
    df_coords_mantain, 
    df_stations,
    # Usamos as colunas arredondadas como chave
    left_on=['start_lat_arred', 'start_lng_arred'],
    right_on=['lat', 'long'],
    how='left',
    suffixes=('', '_start') # Para diferenciar colunas com mesmo nome
)

In [ ]:
df_resultado = pd.merge(
    df_resultado, 
    df_stations,
    # Usamos as colunas arredondadas como chave
    left_on=['end_lat_arred', 'end_lng_arred'],
    right_on=['lat', 'long'],
    how='left',
    suffixes=('', '_end') # Para diferenciar colunas com mesmo nome
)

In [ ]:
df_resultado.head()

In [ ]:
len(df_resultado)

In [ ]:
df_nulls_temp = df_resultado.isna().sum()
df_nulls_temp.sort_values(ascending = False)

Não vale a pena tentar fazer esse processo de verificação de lat e long , é melhor só remover dados faltantes